Some old code not being used anymore. Keeping them here for backup, so can refer to them if needed. 

In [ ]:
### define functions for abundance deviation between MiCRM and EGLV ###

# this is for each community simulation (N consumers), for each temperature
# to be incorporated into the later functions and loops (loop this function for each temp, then for each community simulation)


import numpy as np


def err_eq (C_LV_eq, C_MiCRM_eq, eps=1e-20):

    """

    Compute mean log-ratio error between GLV- and MiCRM-predicted equilibrium consumer abundances 

    
    C_LV_eq: array of equilibrium consumer abundances from GLV (length N) 
    C_MiCRM_eq: array of equilibrium consumer abundances from MiCRM (length N)
    eps: small pseudocount to avoid division by zero (default 1e-7)


    """

    # convert inputs into numpy arrays of floats, for faster computation later

    C_LV = np.asarray(C_LV_eq, dtype=float)
    C_Mi = np.asarray(C_MiCRM_eq, dtype=float)

    # number of consumers 

    N = C_LV.size # length of the array = number of equilibrium consumer values = number of consumers (N) 

    # avoid division by 0 by adding epsilon (pseudocount)
    
    C_LV_safe = np.where(C_LV <= 0, eps, C_LV) # when C_LV is 0 or a tiny negative number, yield eps, else yield C_LV 
    C_Mi_safe = np.where(C_Mi <= 0, eps, C_Mi) # when C_Mi is 0 or a tiny negative number, yield eps, else yield C_Mi

    # ignore species that are extinct in both models (so they don't mess up log ratio average)

    thresh  = 1e-6
    mask = (C_LV_safe > thresh) | (C_Mi_safe > thresh)
    if mask.sum() == 0:
        return np.nan  # extinct 

    # log ratio error for LV vs MiCRM 

    log_ratios = np.log(C_LV_safe[mask] / C_Mi_safe[mask]) # this will be done for each consumer in N. divides 2 arrays (of the same length) element-wise. 
    # only when mask is true (mask is a boolean array), it will calculate the log ratio. if mask is true, it means neither MiCRM / GLV predict extinction. 
    # this way, the log ratio is informative. if both predict it's extinct, then there's no point in calculating that ratio (technically no deviations) 

    equilibrium_error = np.mean(log_ratios) # mean log ratio error across all consumers

    return equilibrium_error # this is the mean log ratio error between GLV and MiCRM for this community simulation at this temperature




In [ ]:
############################# NO GRAPHS - JUST ANALYSIS #############################

import numpy as np
from numpy.random import default_rng
from scipy.integrate import solve_ivp
from numpy import linspace 
import matplotlib.pyplot as plt
from numpy.linalg import norm, eig


# want to first generate parameters for a particular randomly-assembled community
# and then simulate 31 different temperatures for this same community (both MiCRM and EGLV graphs for each temperature) 
# compare how temperature affects the deviation between MiCRM and EGLV graphs
# later can add in equation to quantitatively evaluate deviations, and print that for each temperature
# could also make graph for temperature vs species richness (number of surviving consumers)


rng = default_rng(111)

N = 7
M = 5
L = np.full(N, 0.3) # leakage (this is not temp dependent, as per original MiCRM)

t0, t1 = 0.0, 2.5e10 # from 0 to a large value
tspan = (t0, t1) # time span for integration
x0 = np.concatenate([np.full(N, 0.1), np.full(M, 1)]) # initial conditions for consumers and resources

# Temperature‐dependence parameters
num_temps = 31 # number of temperatures
rho_t = np.array([0.0, 0.0])   # minimal trade‐off
Tr = 273.15 + 10 # reference temperature (10 °C)
Ed = 3.5 

# ─────────────────────────────────────────────────────────────────────────────
# 2) Define a “steady‐state” event for solve_ivp
#    Stops when ‖dx/dt‖ ≤ machine epsilon
# ─────────────────────────────────────────────────────────────────────────────
def mi_crm_rhs(t, x, p):
    return MiCRM_dxx(x, t, p) # MiCRM_dxx is the full RHS of the MiCRM ODE, defined before 

def ss_event(t, x, p): # steady state event 
    return norm(mi_crm_rhs(t, x, p)) - np.finfo(float).eps

ss_event.terminal = True
ss_event.direction = 0

# define steady-state event for GLV as well 

def ss_event_lv(t, x, p_lv):
    return norm(LV_dx(x, t, p_lv)) - np.finfo(float).eps
ss_event_lv.terminal = True
ss_event_lv.direction = 0

# stops integration when system reaches steady state (machine epsilon), so it doesn't run forever

####################
# actually, not sure if I'll be using this s.s. thing (the graph was having truncation issues, idk if it's related to this, so might cut this bit out and not call it. also that sign error.) 
####################



####### TEST OUT DIFFERENT TEMPERATURES #######


structural = generate_params(
    N, M,
    f_u=def_u,      # relative preferences only
    f_m=def_m,      # placeholder
    f_rho=def_rho,
    f_omega=def_omega,
    f_l=def_l,
    # *no* T, Tr, Ed, rho_t here
    L=L,
    T=273.15,   # dummy (This satisfies temp_trait’s requirement that kw contain T, rho_t, Tr, Ed) 
    # since we are using the default def_u and def_m here, they ignore kw, so any T will work 
    # this whole thing will just provide relative preferences (u) and a constant m=1    
    rho_t=rho_t,  # dummy
    Tr=Tr,        # dummy
    Ed=Ed         # dummy
)

# the 'structural' parameters are static and don't change with temperature
# e.g. the def_u only generates relative preferences, not absolute ones
# however the uptake rate (u) will change with temperature. the relative preferences won't. 
# other things like rho and omega also don't change with temperature

# Inspect parameter arrays
# print("Uptake matrix structure u:\n", structural['u']) # should be N x M sized matrix 
# print("Mortality vector structure m:\n", structural['m']) # should be a vector containing M elements 
# print("Leakage tensor l shape:", structural['l'].shape) # should be N x M x M array 




temp_vals = linspace(273.15, 273.15 + 30, num_temps) # 31 temperatures from 0 to 30 degrees C

results = [] # store results for each temperature. it is a list of dictionaries. each temp would produce its own dictionary. 

for T in temp_vals:

    # temp-dependent scalars

    temp_p, B, E, Tp = temp_trait(N, {
        'T': T, 'Tr': Tr, 'Ed': Ed, 'rho_t': rho_t, 'L': L
    })
    temp_p_u = temp_p[:,0]
    temp_p_m = temp_p[:,1]

    # full parameter dictionary for this temp

    pT = {
        **structural,            # brings in u_pref, l, B, E, Tp, L, N, M, etc.
        'u': structural['u'] * temp_p_u[:,None],  # absolute uptake rates. the preference matrix from structural['u'] is multiplied (scaled) by the temperature-dependent uptake rates
        'm': temp_p_m,                            # mortality rates. the preference matrix from structural['m'] is multiplied (scaled) by the temperature-dependent mortality rates
        'lambda': np.sum(structural['l'], axis=2),
        'T': T
    }   
    

    # set up integration range

    t_max_micrm = 5000
    t_max_glv   = 5000

    t_eval_micrm = np.linspace(0, t_max_micrm, 1000)
    t_eval_glv   = np.linspace(0, t_max_glv,   1000)


    # solve MiCRM at this temperature
    

    sol = solve_ivp(
        lambda t, y: MiCRM_dxx(y, t, pT),
        t_span=(0, t_max_micrm),
        y0=x0,
        method='BDF',
        t_eval=t_eval_micrm 
    )

    
    # solve EGLV at this temperature
    
    p_lv = eff_LV_params(pT, sol, verbose=False)

    sol_lv = solve_ivp(
        lambda t, y: LV_dx(y, t, p_lv),
        t_span=(0, t_max_glv), 
        y0=sol.y[:N, 0],
        method='BDF', 
        t_eval=t_eval_glv        
        )
    
    ##### equilibrium deviation calculations #####

    # first collect the equilibrium values for MiCRM and GLV (this is the last value in time series, t1)

    C_MiCRM_eq = sol.y[:N, -1] # equilibrium consumer biomass
    C_LV_eq = sol_lv.y[:N, -1] # equilibrium consumer biomass

    # then compute the mean log-ratio error between GLV- and MiCRM-predicted equilibrium consumer abundances

    """
    Need to check that there are no negative values for equilibrium, or the log ratio may not work 
    e.g. there might be tiny negative numbers due to slight undershoots

    check using the following code (put right before ErrEq) 

    print("MiCRM eq min/max:", C_MiCRM_eq.min(), C_MiCRM_eq.max())
    print("GLV  eq min/max:",    C_LV_eq.min(),    C_LV_eq.max())

    """
    
    print("MiCRM eq min/max:", C_MiCRM_eq.min(), C_MiCRM_eq.max())
    print("GLV  eq min/max:",    C_LV_eq.min(),    C_LV_eq.max())
    
    ErrEq = err_eq(C_LV_eq, C_MiCRM_eq, eps=1e-7) 

    ##### store results as a dictionary ##### 

    results.append(dict(T=T, sol=sol, sol_lv=sol_lv, ErrEq=ErrEq))


# inspect to check that we have 31 error values (one for each temp)

"""

use the below code to check what the 31 error values are: 

errs = [entry['ErrEq'] for entry in results]
print(errs)
print("Got", len(errs), "temperatures, error values.")

"""

###############
# if we want to plot the 62 graphs to visually see it, can just use code from temp_test 
###############

# analyse results for equilibrium abundance

errs = [entry['ErrEq'] for entry in results]
print(errs)
print("Got", len(errs), "temperatures, error values.")

import matplotlib.pyplot as plt

# Convert your temperature array from K to °C
temps_C = temp_vals - 273.15

# Extract the 31 error values
errs = [entry['ErrEq'] for entry in results]

plt.figure()
plt.plot(temps_C, errs)
plt.xlabel("Temperature (°C)")
plt.ylabel("Mean Log-Ratio Error (GLV vs MiCRM)")
plt.title("Equilibrium Abundance Error vs Temperature")
plt.axhline(0, linestyle='--')   # horizontal zero line
plt.tight_layout()
plt.show()





# note: removed diversity metric (# of surviving species) and leading eigenvalue for now
# this file focuses on abundance deviation calculations for GLV vs MiCRM 
# to add back diversity / eigenvalue, paste from temp_test and add back in results.append dictionary 



# problem: kept getting weird graphs with massive spikes. the issue is setting 0 or negative values to epsilon -> log ratio messes up 

In [ ]:
# define functions for diversity 


##### Shannon diversity #####

def shannon (abundance): # abundance should be an array (incl values for all consumers in system) 
    C_shannon = np.asarray(abundance, dtype=float) # convert into numpy arrays of floats 
  
    ### normalise to convert abundance into something on a scale of 0-1 ###

    # total number of 'individuals' (add up all the relative abundances) 
    total_abundance = np.sum(C_shannon) # add up all the elements in this C_shannon array. total_abundance also a single value. 

    pi = C_shannon / total_abundance # pi is  an array. it now converts C_shannon into relative proportions, by dividing each element of C_shannon by the total_abundance value. 

    pi_lnpi = pi[pi > 0] * np.log(pi[pi > 0]) # this is pi * ln(pi). pi_lnpi should also be an array with N elements. 
    # keep only the pi > 0 ones. if pi = 0, will have issues with log. if pi = 0 it won't contribute to Shannon index anyway. 

    H = -np.sum(pi_lnpi) 

    return H 

# note: for Shannon diversity, might need to consider the case where total abundance is 0 (so would be dividing by 0)

# can compare Shannon diversity for 2 samples (GLV vs MiCRM for each temperature)

##### Bray-Curtis dissimilarity ##### 

# defined earlier already, but re-write anyway 


def bray_curtis_dissimilarity(G, M): # G = GLV, M = MiCRM 

    G_array = np.asarray(G, dtype=float) # convert G (abundance of each species predicted by GLV) into array 
    M_array = np.asarray(M, dtype=float) # convert M (abundance of each species predicted by MiCRM) into array 

    G_safe = np.where(G_array < 0, 0, G_array) # if any element (species) of the GLV-predicted array of abundances is less than 0, consider that 0
    M_safe = np.where(M_array < 0, 0, M_array) 

    GM_dissimilarity = np.sum(np.abs(G_safe - M_safe)) / np.sum(G_safe + M_safe) # bray-curtis dissimilarity between GLV and MiCRM predictions 

    return GM_dissimilarity 

# also consider the case where total abundance is 0 (so would be dividing by 0) 



##### simple count of number of overlapping species between GLV and MiCRM #####


def survivors(G, M, thresh=1e-8): # G = GLV, M = MiCRM 

    G_array = np.asarray(G, dtype=float) # convert G (abundance of each species predicted by GLV) into array 
    M_array = np.asarray(M, dtype=float) # convert M (abundance of each species predicted by MiCRM) into array

    # threshold for presence/absence (survivors are the ones coexisting at equilibrium) 

    G_surv = G_array > thresh 
    M_surv = M_array > thresh 

    return G_surv, M_surv 


# G_surv and M_surv are boolean masks of the same shape as original input arrays G, M 
# each element is TRUE if species abundance > threshold, and FALSE otherwise 
# they are themselves not arrays of abundances, but rather are logical filters 





In [ ]:
########################### FOR GRAPHING (62 MINI GRAPHS OF MICRM VS GLV) ###########################



# # test out different temperatures

# import numpy as np
# from numpy.random import default_rng
# from scipy.integrate import solve_ivp
# from numpy import linspace 
# import matplotlib.pyplot as plt
# from numpy.linalg import norm, eig


# # want to first generate parameters for a particular randomly-assembled community
# # and then simulate 31 different temperatures for this same community (both MiCRM and EGLV graphs for each temperature) 
# # compare how temperature affects the deviation between MiCRM and EGLV graphs
# # later can add in equation to quantitatively evaluate deviations, and print that for each temperature
# # could also make graph for temperature vs species richness (number of surviving consumers)


# rng = default_rng(111)

# N = 7
# M = 5
# L = np.full(N, 0.3) # leakage (this is not temp dependent, as per original MiCRM)

# t0, t1 = 0.0, 2.5e10 # from 0 to a large value
# tspan = (t0, t1) # time span for integration
# x0 = np.concatenate([np.full(N, 0.1), np.full(M, 1)]) # initial conditions for consumers and resources

# # Temperature‐dependence parameters
# num_temps = 31 # number of temperatures
# rho_t = np.array([0.0, 0.0])   # minimal trade‐off
# Tr = 273.15 + 10 # reference temperature (10 °C)
# Ed = 3.5 

# # ─────────────────────────────────────────────────────────────────────────────
# # 2) Define a “steady‐state” event for solve_ivp
# #    Stops when ‖dx/dt‖ ≤ machine epsilon
# # ─────────────────────────────────────────────────────────────────────────────
# def mi_crm_rhs(t, x, p):
#     return MiCRM_dxx(x, t, p) # MiCRM_dxx is the full RHS of the MiCRM ODE, defined before 

# def ss_event(t, x, p): # steady state event 
#     return norm(mi_crm_rhs(t, x, p)) - np.finfo(float).eps

# ss_event.terminal = True
# ss_event.direction = 0

# # stops integration when system reaches steady state (machine epsilon), so it doesn't run forever



# ####### TEST OUT DIFFERENT TEMPERATURES #######


# structural = generate_params(
#     N, M,
#     f_u=def_u,      # relative preferences only
#     f_m=def_m,      # placeholder
#     f_rho=def_rho,
#     f_omega=def_omega,
#     f_l=def_l,
#     # *no* T, Tr, Ed, rho_t here
#     L=L,
#     T=273.15,   # dummy (This satisfies temp_trait’s requirement that kw contain T, rho_t, Tr, Ed) 
#     # since we are using the default def_u and def_m here, they ignore kw, so any T will work 
#     # this whole thing will just provide relative preferences (u) and a constant m=1    
#     rho_t=rho_t,  # dummy
#     Tr=Tr,        # dummy
#     Ed=Ed         # dummy
# )

# # the 'structural' parameters are static and don't change with temperature
# # e.g. the def_u only generates relative preferences, not absolute ones
# # however the uptake rate (u) will change with temperature. the relative preferences won't. 
# # other things like rho and omega also don't change with temperature

# # Inspect parameter arrays
# print("Uptake matrix structure u:\n", structural['u']) # should be N x M sized matrix 
# print("Mortality vector structure m:\n", structural['m']) # should be a vector containing M elements 
# print("Leakage tensor l shape:", structural['l'].shape) # should be N x M x M array 




# temp_vals = linspace(273.15, 273.15 + 30, num_temps) # 31 temperatures from 0 to 30 degrees C

# results = [] # store results for each temperature. it is a list of dictionaries. each temp would produce its own dictionary. 

# for T in temp_vals:

#     # temp-dependent scalars

#     temp_p, B, E, Tp = temp_trait(N, {
#         'T': T, 'Tr': Tr, 'Ed': Ed, 'rho_t': rho_t, 'L': L
#     })
#     temp_p_u = temp_p[:,0]
#     temp_p_m = temp_p[:,1]

#     # full parameter dictionary for this temp

#     pT = {
#         **structural,            # brings in u_pref, l, B, E, Tp, L, N, M, etc.
#         'u': structural['u'] * temp_p_u[:,None],  # absolute uptake rates. the preference matrix from structural['u'] is multiplied (scaled) by the temperature-dependent uptake rates
#         'm': temp_p_m,                            # mortality rates. the preference matrix from structural['m'] is multiplied (scaled) by the temperature-dependent mortality rates
#         'lambda': np.sum(structural['l'], axis=2),
#         'T': T
#     }   
    

#     # set up integration range

#     t_max_micrm = 3000
#     t_max_glv   = 3000

#     t_eval_micrm = np.linspace(0, t_max_micrm, 1000)
#     t_eval_glv   = np.linspace(0, t_max_glv,   1000)


#     # solve MiCRM at this temperature
    

#     sol = solve_ivp(
#         lambda t, y: MiCRM_dxx(y, t, pT),
#         t_span=tspan, y0=x0, method='BDF',
#         t_eval=t_eval_micrm           
#         )

    
#     # solve EGLV at this temperature
    
#     p_lv = eff_LV_params(pT, sol, verbose=False)

#     sol_lv = solve_ivp(
#         lambda t, y: LV_dx(y, t, p_lv),
#         t_span=(0, t1), y0=sol.y[:N, 0],
#         method='BDF',
#         t_eval=t_eval_glv          
#         )
    
#     ##### store other interesting summary info #####

#     # equilibrium values

#     bm_eq = sol.y[:N, -1] # equilibrium consumer biomass 
#     survivors = np.where(bm_eq > 1e-7)[0]   # indices of extant consumers. 1.0e-7 is a threshold for extinction
#     N_s = survivors.size # number of survivors (equilibrium community richness)

#     # leading eigenvalue

#     LV_jac = eff_LV_jac(p_lv, sol)
#     eigvals = eig(LV_jac)[0]
#     leading = eigvals[np.argmax(np.real(eigvals))] # eigenvalue with largest real part (dominant/leading eigenvalue)    

#     ##### store results as a dictionary ##### 

#     results.append(dict(T=T, sol=sol, sol_lv=sol_lv, N_s=N_s, leading=leading))



# # plot results as mini plots 
# # it should show mini plots for 31 temperatures, each having MiCRM and EGLV side by side 

# nT = num_temps
# cols = 2

# # can set different t_max for micrm / glv
# # e.g. glv might take longer to equilibrate 
# # this will help us visualise deviations in final equilibrium values (if that's what we're interested in)

# fig, axes = plt.subplots(nT, cols, figsize=(6, nT*1.5), sharex=True, sharey=True)

# for idx, run in enumerate(results):
#     T = run['T'] - 273.15

#     # --- MiCRM panel ---
#     sol = run['sol']
#     ts = sol.t
#     mask = ts <= t_max_micrm
#     ts, Y = ts[mask], sol.y[:, mask]
#     ax1 = axes[idx, 0]
#     for i in range(N):
#         ax1.plot(ts, Y[i], lw=1)
#     for j in range(M):
#         ax1.plot(ts, Y[N+j], '--', lw=1)
#     ax1.set_xlim(0, t_max_micrm)
#     if idx == 0:
#         ax1.set_title("MiCRM")
#     if idx < (nT - 1):
#         ax1.set_xticks([])

#     # --- GLV panel ---
#     sol_lv = run['sol_lv']
#     ts_lv = sol_lv.t
#     mask_lv = ts_lv <= t_max_glv
#     ts_lv, Y_lv = ts_lv[mask_lv], sol_lv.y[:, mask_lv]
#     ax2 = axes[idx, 1]
#     for i in range(N):
#         ax2.plot(ts_lv, Y_lv[i], lw=1)
#     ax2.set_xlim(0, t_max_glv)
#     if idx == 0:
#         ax2.set_title("GLV")
#     if idx < (nT - 1):
#         ax2.set_xticks([])

#     # label the left of each row
#     ax1.set_ylabel(f"{T:.0f}°C")

# # shared labels
# fig.supxlabel("Time")
# fig.supylabel("Abundance")
# plt.tight_layout()
# plt.show()





In [ ]:
# random functions 

# ————————————————————————————————————————————————
# Simple distance/similarity metrics
# ————————————————————————————————————————————————

def cosine_similarity(A, B):
    """
    Cosine similarity between 1D vectors A and B.
    """
    return np.dot(A, B) / (norm(A) * norm(B))

# bray-curtis dissimilarity function: have modified it later on 

def euclidean_distance(A, B):
    """
    Euclidean distance between 1D vectors A and B.
    """
    return np.linalg.norm(A - B)

# ————————————————————————————————————————————————
# AIC for fitted models
# ————————————————————————————————————————————————

def AIC(model, n):
    """
    Akaike Information Criterion.
      - model.residuals(): array of residuals
      - model.coef(): array of fitted parameters
      - n: number of data points
    """
    residuals = model.residuals()  # user‐provided method
    RSS = np.sum(residuals ** 2)
    k = len(model.coef())
    return n * math.log(RSS / n) + 2 * k

In [ ]:
"""
10/05/2025

need to clean up this section of code. here is a backup of the current version. 

"""

############################# PRODUCING ACTUAL SIMULATION GRAPHS #############################

import numpy as np
from numpy.random import default_rng
from scipy.integrate import solve_ivp
from numpy import linspace 
import matplotlib.pyplot as plt
from numpy.linalg import norm, eig

"""
want to first generate parameters for a particular randomly-assembled community
and then simulate 31 different temperatures for this same community (both MiCRM and EGLV graphs for each temperature) 
compare how temperature affects the deviation between MiCRM and EGLV graphs
"""


rng = default_rng(111)

N = 7
M = 5
L = np.full(N, 0.3) # leakage (this is not temp dependent, as per original MiCRM)

t0, t1 = 0.0, 2.5e10 # from 0 to a large value
tspan = (t0, t1) # time span for integration
x0 = np.concatenate([np.full(N, 0.1), np.full(M, 1)]) # initial conditions for consumers and resources

# Temperature‐dependence parameters
num_temps = 31 # number of temperatures
rho_t = np.array([0.0, 0.0])   # minimal trade‐off
Tr = 273.15 + 10 # reference temperature (10 °C)
Ed = 3.5 

# ─────────────────────────────────────────────────────────────────────────────
# 2) Define a “steady‐state” event for solve_ivp
#    Stops when ‖dx/dt‖ ≤ machine epsilon
# ─────────────────────────────────────────────────────────────────────────────
def mi_crm_rhs(t, x, p):
    return MiCRM_dxx(x, t, p) # MiCRM_dxx is the full RHS of the MiCRM ODE, defined before 

def ss_event(t, x, p): # steady state event 
    return norm(mi_crm_rhs(t, x, p)) - np.finfo(float).eps

ss_event.terminal = True
ss_event.direction = 0

# define steady-state event for GLV as well 

def ss_event_lv(t, x, p_lv):
    return norm(LV_dx(x, t, p_lv)) - np.finfo(float).eps
ss_event_lv.terminal = True
ss_event_lv.direction = 0

# stops integration when system reaches steady state (machine epsilon), so it doesn't run forever

####################
# actually, not sure if I'll be using this s.s. thing (the graph was having truncation issues, idk if it's related to this, so might cut this bit out and not call it. also that sign error.) 
####################



####### TEST OUT DIFFERENT TEMPERATURES #######


structural = generate_params(
    N, M,
    f_u=def_u,      # relative preferences only
    f_m=def_m,      # placeholder
    f_rho=def_rho,
    f_omega=def_omega,
    f_l=def_l,
    # *no* T, Tr, Ed, rho_t here
    L=L,
    T=273.15,   # dummy (This satisfies temp_trait’s requirement that kw contain T, rho_t, Tr, Ed) 
    # since we are using the default def_u and def_m here, they ignore kw, so any T will work 
    # this whole thing will just provide relative preferences (u) and a constant m=1    
    rho_t=rho_t,  # dummy
    Tr=Tr,        # dummy
    Ed=Ed         # dummy
)

# the 'structural' parameters are static and don't change with temperature
# e.g. the def_u only generates relative preferences, not absolute ones
# however the uptake rate (u) will change with temperature. the relative preferences won't. 
# other things like rho and omega also don't change with temperature

# Inspect parameter arrays
# print("Uptake matrix structure u:\n", structural['u']) # should be N x M sized matrix 
# print("Mortality vector structure m:\n", structural['m']) # should be a vector containing M elements 
# print("Leakage tensor l shape:", structural['l'].shape) # should be N x M x M array 




temp_vals = linspace(273.15, 273.15 + 30, num_temps) # 31 temperatures from 0 to 30 degrees C

results = [] # store results for each temperature. it is a list of dictionaries. each temp would produce its own dictionary. 

for T in temp_vals:

    # temp-dependent scalars

    temp_p, B, E, Tp = temp_trait(N, {
        'T': T, 'Tr': Tr, 'Ed': Ed, 'rho_t': rho_t, 'L': L
    })
    temp_p_u = temp_p[:,0]
    temp_p_m = temp_p[:,1]

    # full parameter dictionary for this temp

    pT = {
        **structural,            # brings in u_pref, l, B, E, Tp, L, N, M, etc.
        'u': structural['u'] * temp_p_u[:,None],  # absolute uptake rates. the preference matrix from structural['u'] is multiplied (scaled) by the temperature-dependent uptake rates
        'm': temp_p_m,                            # mortality rates. the preference matrix from structural['m'] is multiplied (scaled) by the temperature-dependent mortality rates
        'lambda': np.sum(structural['l'], axis=2),
        'T': T
    }   
    

    # set up integration range

    t_max_micrm = 5000
    t_max_glv   = 5000

    t_eval_micrm = np.linspace(0, t_max_micrm, 1000)
    t_eval_glv   = np.linspace(0, t_max_glv,   1000)


    # solve MiCRM at this temperature
    

    sol = solve_ivp(
        lambda t, y: MiCRM_dxx(y, t, pT),
        t_span=(0, t_max_micrm),
        y0=x0,
        method='BDF',
        t_eval=t_eval_micrm 
    )

    
    # solve EGLV at this temperature
    
    p_lv = eff_LV_params(pT, sol, verbose=False)

    sol_lv = solve_ivp(
        lambda t, y: LV_dx(y, t, p_lv),
        t_span=(0, t_max_glv), 
        y0=sol.y[:N, 0],
        method='BDF', 
        t_eval=t_eval_glv        
        )
    
    ##### deviation calculations #####

    # first collect the equilibrium values for MiCRM and GLV (this is the last value in time series, t1)
    C_MiCRM_eq = sol.y[:N, -1] # equilibrium consumer biomass
    C_LV_eq = sol_lv.y[:N, -1] # equilibrium consumer biomass

       
    # equilibrium abundance deviation 
    ErrEqAb, overlap = err_eq_and_overlap(C_LV_eq, C_MiCRM_eq)

    # trajectory deviation  
    times      = sol.t                   # shape (T,). this defines the time array to be investigated in trajectory deviations 
    C_Mi_traj  = sol.y[:N, :]            # shape (N, T). this is the MiCRM trajectory, to be analysed in trajectory deviations 
    C_LV_traj  = sol_lv.y[:N, :]         # shape (N, T). this is the GLVM trajectory, to be analysed in trajectory deviations 
    err_t, overlap_t = err_time_series(times, C_LV_traj, C_Mi_traj) # first get the time series 
    j_eq = estimate_teq(times, sol, sol_lv, pT, p_lv, tol=1e-6, window=5) # find equilibrium time 
    times_crop = times[: j_eq + 1] # crop time array to equilibrium 
    err_crop   = err_t[: j_eq + 1] # crop trajectory error array to equilibrium 
    Err_traj = integrate_err(times_crop, err_crop) # integrate over [0, t_eq] instead of [0, t_max]

    # diversity deviation
    jaccard = jaccard_index(C_LV_eq, C_MiCRM_eq, thresh=1e-6)
    sh_LV = shannon(C_LV_eq)
    sh_Mi = shannon(C_MiCRM_eq)
    bc = bray_curtis_dissimilarity(C_LV_eq, C_MiCRM_eq)
    
    ##### store results as a dictionary ##### 

    results.append(dict(T=T, 
                        sol=sol, 
                        sol_lv=sol_lv, 
                        ErrEqAb=ErrEqAb, 
                        overlap=overlap,
                        ErrTraj=Err_traj,
                        jaccard=jaccard,
                        sh_LV=sh_LV,
                        sh_Mi=sh_Mi,
                        bray_curtis=bc
                        ))

# analyse results for equilibrium abundance

# extract values for plotting 
temps_C = temp_vals - 273.15 # Convert temperature array from K to °C
errs     = [r['ErrEqAb'] for r in results]
overlaps = [r['overlap']  for r in results]
errtraj = [r['ErrTraj']  for r in results]
jaccards = [r['jaccard'] for r in results]
sh_lvs   = [r['sh_LV']   for r in results]
sh_mis   = [r['sh_Mi']   for r in results]
bcs = [r['bray_curtis']   for r in results]



######################################### PLOTTING #############################################


import matplotlib.pyplot as plt

# plot abundance deviation at equilibrium 
plt.figure()
plt.plot(temps_C, errs)
plt.xlabel("Temperature (°C)")
plt.ylabel("Mean Log-Ratio Error (GLV vs MiCRM)")
plt.title("Equilibrium Abundance Error vs Temperature")
plt.axhline(0, linestyle='--')   # horizontal zero line
plt.tight_layout()
plt.show()

# plot number of species overlap for GLV/MiCRM at equilibrium (accuracy in terms of diversity predictions)
plt.figure()
plt.plot(temps_C, overlaps, marker='s', label='Overlap Count')
plt.xlabel("Temperature (°C)")
plt.ylabel("Number of Shared Survivors")
plt.title("Shared Species Count vs Temperature")
plt.ylim(-0.5, N + 0.5)  # N is total species, e.g. 7
plt.legend()
plt.tight_layout()
plt.show()

# plot trajectory deviation 
plt.figure()
plt.plot(temps_C, errtraj)
plt.xlabel("Temperature (°C)")
plt.ylabel("Trajectory Deviation (GLV vs MiCRM)")
plt.title("Trajectory Error vs Temperature")
plt.axhline(0, linestyle='--')   # horizontal zero line
plt.tight_layout()
plt.show()

# Temperature vs Jaccard
plt.figure()
plt.plot(temps_C, jaccards, marker='o')
plt.xlabel("Temperature (°C)")
plt.ylabel("Jaccard Index (GLV vs MiCRM)")
plt.title("Jaccard vs Temperature")
plt.ylim(-0.05, 1.05)
plt.tight_layout()
plt.show()

# Temperature vs Shannon diversity (two lines)
plt.figure()
plt.plot(temps_C, sh_lvs, marker='s', label="GLV Shannon")
plt.plot(temps_C, sh_mis, marker='^', label="MiCRM Shannon")
plt.xlabel("Temperature (°C)")
plt.ylabel("Shannon Diversity H'")
plt.title("Shannon Diversity vs Temperature")
plt.legend()
plt.tight_layout()
plt.show()

# Bray-Curtis (for community composition - incorporating both diversity + abundance)

plt.figure()
plt.plot(temps_C, bcs, marker='o')
plt.xlabel("Temperature (°C)")
plt.ylabel("Bray–Curtis Dissimilarity")
plt.title("Bray–Curtis Dissimilarity vs Temperature")
plt.ylim(-0.05, 1.05)
plt.tight_layout()
plt.show()



In [ ]:
"""
this code worked before, but ran into issues after changing some other functions to vectorised form (to improve efficiency and reduce run time)
so had to modify the bauer fike bound function in the actual file
"""

def bauer_fike_bound(J_micrm, J_glv, threshold=1e-7):
    # pick feasible species
    bm = np.diag(J_micrm)[:J_glv.shape[0]]  # diag entries proportional to C_i
    feas  = np.where(bm > threshold)[0]

    if len(feas) == 0:
        raise ValueError("No feasible species found above threshold.")
    
    Jr    = J_micrm[np.ix_(feas, feas)]
    Jg    = J_glv[np.ix_(feas, feas)]
    ΔJ    = Jr - Jg
    normΔ = np.linalg.norm(ΔJ, ord='fro')
    V     = np.linalg.eig(Jr)[1]
    condV = np.linalg.norm(V, 2) * np.linalg.norm(np.linalg.inv(V), 2)
    return condV * normΔ